# Criação das tabelas dimensão e fatos

In [0]:
from pyspark.sql.functions import monotonically_increasing_id, row_number, split, explode
from pyspark.sql import Window

In [0]:
df_goodreads = spark.read.format('delta').load('abfss://livros@recomendador.dfs.core.windows.net/silver/goodreads')
df_kindle = spark.read.format('delta').load('abfss://livros@recomendador.dfs.core.windows.net/silver/kindle')

> Juntar as tabelas com as informações que queremos para análise

In [0]:
df_goodreads.printSchema()

In [0]:
df_kindle.printSchema()

In [0]:
df_join = df_goodreads.join(df_kindle, df_goodreads.title == df_kindle.title, 'inner') \
                    .drop(df_goodreads.title, df_goodreads.author, df_goodreads.rating, df_kindle.reviews)

In [0]:
df_join.toPandas().head(1)

## Exclusão das variáveis

In [0]:
del df_goodreads, df_kindle

In [0]:
df_join.groupBy(df_join.title) \
        .count() \
        .orderBy('count', ascending=False) \
        .show(10)

In [0]:
df_join.columns

## Criação das tabelas e dimensões

> Tabela _dimensão autor_

In [0]:
w = Window.orderBy(df_join.author)
df_dim_autor = df_join.select(row_number().over(w).alias('id'), (df_join.author).alias('autor')) \
                        .distinct()

> Tabela _dimensão livro_

In [0]:
w = Window.orderBy(df_join.title)
df_dim_livro = df_join.select(row_number().over(w).alias('id'), df_join.isbn, (df_join.title).alias('titulo'), (df_join.bookformat).alias('formato_do_livro')) \
                        .distinct()

> Tabelas _ponte grupo gênero_, _dimensão grupo_ e _dimensão gênero_

In [0]:
df_join.select(df_join.category_id, df_join.category_name, df_join.genre).toPandas().head(10)

In [0]:
df_bg_grupo_genero = df_join.select(df_join.genre).distinct()
df_bg_grupo_genero = df_bg_grupo_genero \
                                            .withColumn('lista_de_generos', split(df_bg_grupo_genero.genre, ',')) \
                                            .withColumn('id_grupo', monotonically_increasing_id()+1) \
                                            .withColumn('genero', explode('lista_de_generos'))

In [0]:
df_dim_genero = df_bg_grupo_genero.select('genero') \
                                .distinct() \
                                .withColumn('id', monotonically_increasing_id()+1) \
                                .select('id', 'genero')

In [0]:
df_bg_grupo_genero = df_bg_grupo_genero.join(df_dim_genero, df_bg_grupo_genero.genero == df_dim_genero.genero, 'inner') \
                                        .drop(df_dim_genero.genero) \
                                        .dropDuplicates(['id', 'id_grupo']) \
                                        .select('id_grupo', 'lista_de_generos', (df_bg_grupo_genero.genre).alias('generos'), 'genero', 'id')

In [0]:
df_fatos = df_join.join(df_bg_grupo_genero, df_join.genre == df_bg_grupo_genero.generos, 'inner') \
                .drop(df_bg_grupo_genero.generos, df_bg_grupo_genero.lista_de_generos, df_join.genre)

df_fatos = df_fatos.select((df_fatos.id).alias('id_livro'), 'id_grupo', (df_fatos.pages).alias('numero_de_paginas'), (df_fatos.stars).alias('numero_de_estrelas'), (df_fatos.reviews).alias('numero_de_avaliacoes'), (df_fatos.totalratings).alias('total_de_avaliacoes'), (df_fatos.price).alias('preco'), (df_fatos.isKindleUnlimited).alias('esta_no_kindle_unlimited'), (df_fatos.isGoodReadsChoice).alias('esta_no_goodreads_choice'), (df_fatos.publishedDate).alias('data_de_publicacao'), df_fatos.title, df_fatos.author)

In [0]:
del w, df_join

In [0]:
df_dim_grupo = df_bg_grupo_genero.select((df_bg_grupo_genero.id_grupo).alias('id'), df_bg_grupo_genero.lista_de_generos)

In [0]:
df_bg_grupo_genero = df_bg_grupo_genero.select('id_grupo', (df_bg_grupo_genero.id).alias('id_genero'))

> Tabela _fatos_

In [0]:
df_fatos = df_fatos.join(df_dim_livro, df_fatos.title == df_dim_livro.titulo, 'inner') \
                    .drop(df_fatos.title, df_dim_livro.titulo)

df_fatos = df_fatos.select((df_fatos.id).alias('id_livro'), 'id_grupo', 'numero_de_paginas', 'numero_de_estrelas','numero_de_avaliacoes', 'total_de_avaliacoes', 'preco', 'esta_no_kindle_unlimited', 'esta_no_goodreads_choice', 'data_de_publicacao', 'author')

In [0]:
df_fatos = df_fatos.join(df_dim_autor, df_fatos.author == df_dim_autor.autor, 'inner') \
                    .drop(df_fatos.author, df_dim_autor.autor)

df_fatos = df_fatos.select('id_livro', (df_fatos.id).alias('id_autor'), 'id_grupo', 'numero_de_paginas', 'numero_de_estrelas','numero_de_avaliacoes', 'total_de_avaliacoes', 'preco', 'esta_no_kindle_unlimited', 'esta_no_goodreads_choice', 'data_de_publicacao')

## Salvação dos dados na camada _gold_

In [0]:
df_dim_livro.write.mode('overwrite').format('delta').save('abfss://livros@recomendador.dfs.core.windows.net/gold/dim_livro')
df_dim_autor.write.mode('overwrite').format('delta').save('abfss://livros@recomendador.dfs.core.windows.net/gold/dim_autor')
df_dim_grupo.write.mode('overwrite').format('delta').save('abfss://livros@recomendador.dfs.core.windows.net/gold/dim_grupo_genero')
df_dim_genero.write.mode('overwrite').format('delta').save('abfss://livros@recomendador.dfs.core.windows.net/gold/dim_genero')
df_bg_grupo_genero.write.mode('overwrite').format('delta').save('abfss://livros@recomendador.dfs.core.windows.net/gold/bg_grupo_genero')
df_fatos.write.mode('overwrite').format('delta').save('abfss://livros@recomendador.dfs.core.windows.net/gold/fatos_livro')

## Exclusão das variáveis

In [0]:
del df_dim_autor, df_dim_grupo, df_dim_genero, df_dim_livro, df_fatos, df_bg_grupo_genero